In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [3]:
import undetected_chromedriver as uc

In [4]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'sec-ch-ua': 'Google Chrome;v="89", "Chromium";v="89", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,fr;q=0.5'}


In [7]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.52


In [8]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [9]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]

    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = webdriver.ChromeOptions()

    # Going undercover:
    option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
        
    return option

In [10]:
conn = mysql.connector.connect(
    host='103.56.158.31',
    port = 3306,
    user= 'tuyendungUser',
    password='sinhvienBK',
    database= 'ThongTinTuyenDung'
)
cursor = conn.cursor()

In [11]:

sql = 'INSERT IGNORE INTO Stg_ThongTin_raw(Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong) VALUES (%s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s, %s, %s, %s,%s)'

In [12]:
sql_link = 'SELECT Link FROM Stg_ThongTin_raw where Web =\'123jobs\''

In [13]:
cursor.execute(sql_link)
result = cursor.fetchall()
remove_url_list = [row[0] for row in result]

In [14]:
def extract(url):
    driver.get(url)
    Web = '123jobs'
    Nganh = 'IT'
    Link = url
    TenCV = driver.find_element(By.CSS_SELECTOR,'h1.job-title strong').text
    try:
        CongTy = driver.find_element(By.CSS_SELECTOR,'[class="col-md-9 content-group box-apply-top js-item-job"] p').text
    except:
        CongTy = ""
    TinhThanh = driver.find_elements(By.CSS_SELECTOR,'[class="item text-black"]')[0].text
    Luong = driver.find_element(By.CSS_SELECTOR,'[class = "item salary"]').text
    LoaiHinh = driver.find_elements(By.CSS_SELECTOR,'[class="item text-black"]')[1].text
    KinhNghiem = driver.find_elements(By.CSS_SELECTOR,'[class="item text-black"]')[2].text
    CapBac = driver.find_elements(By.CSS_SELECTOR,'[class="item time-expiry-date"]')[1].text
    HanNopCV = driver.find_elements(By.CSS_SELECTOR,'[class="item time-expiry-date"]')[0].text
    try:
        YeuCau = driver.find_element(By.CSS_SELECTOR, '[class="collslap"]').find_elements(By.CSS_SELECTOR,'[class="content-group"]')[1].text
    except:
        YeuCau = ""
    try:
        MoTa = driver.find_element(By.CSS_SELECTOR, '[class="collslap"]').find_elements(By.CSS_SELECTOR,'[class="content-group"]')[0].text
    except:
        MoTa = ""
    try:
        PhucLoi = driver.find_element(By.CSS_SELECTOR, '[class="collslap"]').find_elements(By.CSS_SELECTOR,'[class="content-group"]')[2].text
    except:
        PhucLoi = ""
    SoLuong = driver.find_elements(By.CSS_SELECTOR,'[class="item text-black"]')[3].text
    cursor.execute(sql, (Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong))
    conn.commit()

    

In [15]:
job_urls = []
def it(url_first):
    driver.get(url_first)
    job_page_url_list = driver.find_elements(By.CSS_SELECTOR, '[class="job__list-item-title"]')
    for url in job_page_url_list:
        link = url.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        job_urls.append(link)
    next = driver.find_element(By.CSS_SELECTOR,'[rel="Next"]').get_attribute("href")
    if "page=11" in next:
        next = None
    if next is not None:
        it(next)

In [16]:
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
it("https://123job.vn/nganh-nghe/vi%E1%BB%87c-l%C3%A0m-it-ph%E1%BA%A7n-m%E1%BB%81m?cat=IT+ph%E1%BA%A7n+m%E1%BB%81m&sort=new&cat_name=IT+ph%E1%BA%A7n+m%E1%BB%81m&page=1")
driver.quit()
print("Số url cào về: ", len(job_urls))
for job_url in remove_url_list:
    if job_url in job_urls:
        job_urls.remove(job_url)
print("Số lượng url mới đã lấy được: ", len(job_urls), " url")
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
if len(job_urls) > 0:
    for i in range(len(job_urls)):
        print("Đang xử lý: ", job_urls[i])
        extract(job_urls[i])
        print("Complete")
else:
    print("Không có tin mới")
driver.quit()
cursor.close()
conn.close()

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Số url cào về:  300
Số lượng url mới đã lấy được:  205  url


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Đang xử lý:  https://123job.vn/viec-lam/nam-tu-liem-junior-middle-W9pXnOvx9p?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/cau-giay-junior-game-dev-vrzBVejyr0?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/java-fullstack-developer-spring-up-to-2600-P9YVyQWjra?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/it-officer-LqkEO8eb93?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/embedded-programming-pngineer-advanced-driver-assistance-systems-adas-ArX8zg4ZrV?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/nhan-vien-lap-trinh-cnc-catia-29ALZPvaDJ?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/devops-engineer-azure-39JmzykBDm?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/it-infra-security-staff-n9RwdQnYrb?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/mespi-engineer-LqkEO8yO93?codePosition=C1
Complete
Đang xử lý:  https://123job.vn/viec-lam/nhan-vien-